<a href="https://colab.research.google.com/github/TasnimTamanna02/Langchain_Agent_Restaurant_name_and_Menu_Suggestion/blob/main/Langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
!pip install groq
!pip install langchain langchain-community langchain-core sentence-transformers faiss-cpu unstructured python-docx
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━

In [7]:
import sys
import os

module_path = '/content/drive/MyDrive/Colab Notebooks/AI Practice/ReAct Agent/GroqModule'
file_path = os.path.join(module_path, 'GroqSetup.py')

if os.path.exists(file_path):
    if module_path not in sys.path:
        sys.path.append(module_path)
    try:
        from GroqSetup import api_key, run_llm_prompt, llm
        print("Successfully imported `api_key`,`run_llm_prompt`,`llm` from GroqSetup")
    except (ModuleNotFoundError, AttributeError) as e:
        print("Import failed:", e)
else:
    print("GroqSetup.py not found at expected location.")


Successfully imported `api_key`,`run_llm_prompt`,`llm` from GroqSetup


In [8]:
# #For force reload
# import importlib
# import GroqSetup
# importlib.reload(GroqSetup)
# from GroqSetup import api_key, run_llm_prompt, llm

In [9]:
response = run_llm_prompt(prompt="Explain entropy in simple terms.")
print(response)

/content/drive/MyDrive/Colab Notebooks/AI Practice/ReAct Agent/GroqModule/GroqSetup.py:20: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([message])


Entropy is a concept that can be a bit tricky, but I'll try to break it down in simple terms:

**Entropy is a measure of disorder or randomness.**

Imagine you have a tidy room with everything in its place. This is a state of low entropy, where everything is organized and structured.

Now, imagine that room after a big party. There are clothes scattered all over the floor, empty bottles and cups everywhere, and the furniture is disarranged. This is a state of high entropy, where everything is disorganized and random.

In the same way, entropy can be applied to physical systems, like a cup of hot coffee. When the coffee is freshly made, the molecules are organized and moving in a structured way, which is a state of low entropy. But as the coffee cools down, the molecules start moving randomly and become disorganized, which is a state of high entropy.

**The key idea is that entropy always increases over time.** This means that things tend to become more disorganized and random as time p

### Individual chains

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_name= PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fancy name. Only one name and nothing else."
)

name_chain = prompt_name | llm

name = name_chain.invoke({"cuisine": "Mexican"})
print(name.content)

"El Fuego de Oro"


In [17]:
prompt_menu= PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest some menu items for {restaurant_name}. Return it as a comma-separated list."
)
food_chain = prompt_menu | llm

In [18]:
menu = food_chain.invoke({"restaurant_name": name})
print(menu.content)

Carne Asada Tacos, Chicken Fajitas, Spicy Shrimp Quesadilla, Mexican Street Corn, Chiles Rellenos, Sopes de Chorizo, Empanadas, Guacamole Fresco.


### RunnableSequence

In [23]:
from langchain.schema.runnable import RunnableSequence
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [26]:
name_chain = prompt_name | llm | output_parser

def inject_name(cuisine):
    name = name_chain.invoke({"cuisine": cuisine})
    return name

prompt_overall = PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest some menu items for {restaurant_name}. Return it as a comma-separated list."
)

overall_chain = RunnableLambda(inject_name) | prompt_overall | llm | output_parser

restaurant_name = inject_name("Indian")
response = overall_chain.invoke("Indian")

print("Restaurant Name:", restaurant_name)
print("Menu:", response)

Restaurant Name: Taj Mahal Palace Bistro
Menu: Chicken Tikka Masala, Palak Paneer, Tandoori Chicken, Garlic Naan, Saag Aloo, Lamb Korma, Vegetable Biryani, Raita, Papadum, Mattar Paneer, Chicken Tandoori, Basmati Rice, Gulab Jamun.
